In [ ]:
# This file is derived from /crewaitest4/ManagementConsultant_DigitalTwin_GPT3.5-withCustomtool_new_managerllm.ipynb

In [1]:
import os
import hashlib
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, DirectoryReadTool, FileReadTool
from crewai_tools import BaseTool
from langchain_openai import ChatOpenAI
from openai import OpenAI

# Set your API keys (replace 'Your Key' with actual keys)
os.environ["SERPER_API_KEY"] = "your_api_key_here"  # Replace with your actual Serper API key
os.environ["OPENAI_API_KEY"] = "your_api_key_here"  # Replace with your actual OpenAI API key
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo' #preferred model, but darn expensive!


# Initialize the tools
search_tool = SerperDevTool()
directory_read_tool = DirectoryReadTool(directory='company_docs')
file_read_tool = FileReadTool()

#Initialize the custom tool
from tools import DocumentSummarizationTool
client =OpenAI()
summarization_tool = DocumentSummarizationTool()

# Agents....Agents...Agents...

# Define the Data Analysis Agent
data_analyst_agent = Agent(
    role="Data Analyst/Scientist",
    goal="Collect, process, and analyze data to generate insightful and actionable study reports for existing and new clients.",
    backstory="You're tasked with supporting LN Consulting's comprehensive strategy reports for existing and new clients. "
              "You gather and clean data from various sources relevant to market and competitor analysis, "
              "perform statistical analyses and data modeling, and create visualizations, wherever possible to interpret data trends. "
              "You collaborate with the Research Specialist and Lead Consultant to integrate your findings into strategic recommendations.",
    #allow_delegation=False,
    tools=[search_tool],
    verbose=True
)


# Define the Research Specialist Agent
research_specialist_agent = Agent(
    role="Research Specialist",
    goal="Conduct thorough research to gather qualitative and quantitative information relevant to study topics for existing and new clients.",
    backstory="You're responsible for supporting LN Consulting's comprehensive strategy reports by conducting in-depth research for existing and new clients. "
              "You review relevant literature, studies, and data sources on market trends and competition, "
              "synthesize this information, and collaborate with the Data Analyst to integrate research findings with data analysis. "
              "Your research provides the foundation for strategic recommendations and report writing.",
    #allow_delegation=False,
    tools=[search_tool, directory_read_tool, file_read_tool],
    verbose=True
)

# Define the Lead Consultant Agent (Consultant Digital Twin)
lead_consultant_agent = Agent(
    role="Lead Consultant",
    goal="Oversee the entire study report generation process, ensuring timely delivery and high-quality output for existing and new clients. "
         "Identify solutions in line with desired outcome in {client_problem} and analysed problem statement from analysis_agent. "
         "You will map these solutions to the services offered by LN consulting, identify the relevant case studies, and generate a comprehensive report.",
    backstory="You're the driving force behind LN Consulting's comprehensive strategy reports for existing and new clients. "
              "You plan and coordinate all phases of the report generation process, manage timelines, resources, and budgets, "
              "and communicate with clients to understand their requirements and provide updates. "
              "You ensure that all team members are aligned and meeting project milestones. "
              "Additionally, you have a knack for compiling insightful reports that provide actionable solutions based on analysed problem statement mapping to desired outcome and timeline. "
              "Your reports are known for their clarity, depth, and practical recommendations.",
    allow_delegation=False,
    tools=[ directory_read_tool, file_read_tool ],
    verbose=True
)
# Define the agent
message_drafter = Agent(
    role='Message Drafter',
    goal='Create concise and effective message drafts for email, Twitter DM, and LinkedIn.',
    verbose=True,
    memory=True,
    backstory=(
        "As an expert communicator, you excel at crafting concise and impactful messages tailored for various platforms, peppering with relevant a movie quote."
    ),
    tools=[summarization_tool],
    allow_delegation=False
)


# Task for validating the client's problem statement
data_analysis_task = Task(
    description=(
        "1. Collect and clean data from various sources relevant to the client's market and competition using {client_problem} as context.\n"
        "2. Perform statistical analyses and data modeling to identify trends and insights wherever possible.\n"
        "3. Create visualizations to interpret data trends and provide data-driven insights wherever possible.\n"
        "4. Collaborate with the Research Specialist and Lead Consultant to integrate findings into strategic recommendations."
    ),
    expected_output="A comprehensive data analysis report with deep insights.",
    agent=data_analyst_agent,
    output_file='problem_analysis.md'
)

# Task for understanding the client's problem
research_task = Task(
    description=(
        "1. Identify and review relevant literature, studies, and data sources on market trends and competition related to {client_problem}.\n"
        "2. Synthesize information to provide comprehensive background and context for strategic recommendations.\n"
        "3. Collaborate with the Data Analyst to integrate research findings with data analysis.\n"
        "4. Write and edit sections of the report to ensure clarity and coherence."
    ),
    expected_output="A detailed research document with synthesized information and contextual background.",
    agent=research_specialist_agent,
    output_file='research_doc.md'
)

# Task for vetting research and generating a report
lead_consultant_task = Task(
    description=(
        "1. Plan and coordinate all phases of the report generation process for existing and new clients.\n"
        "2. Manage timelines, resources, and budgets to ensure project efficiency.\n"
        "3. Communicate with clients to understand their requirements in {client_problem} and provide regular updates.\n"
        "4. Ensure all team members are aligned and meeting project milestones.\n"
        "5. Research possible solutions for the identified issues in {client_problem} using LN Consulting Service Delivery process and methodologies.\n"
        "6. List and detail solutions that match desired outcome and timeline in {client_problem}.\n"
        "7. Map solutions in line with services offered by LN consulting.\n"
        "8. Create a brief sample report as prepared by LN that outlines these solutions, including strategic recommendations and actionable insights and relevant case studies from LN Consulting.\n"
    ),
    expected_output="A 2-page sample report in markdown format, including a deep understanding of client requirements in {client_problem} with "
                    "teaser of possible solutions, how LN Consulting can deliver, and relevant case studies. ",
    tools=[directory_read_tool, file_read_tool],
    agent=lead_consultant_agent,
    output_file='sample_report.md'
)

# Define the summarization task
summarization_task = Task(
    description=(
        "Summarize the content of the provided document file 'sample_report.md'. Ensure the summary is concise and covers key points."
    ),
    expected_output='A summary of the document content.',
    tools=[summarization_tool],
    agent=message_drafter,
    input_file='./sample_report.md',  # Ensure this path is correct
    output_file='summary_output.txt'  # Specify the output file for the summary
)

# Define the message drafting task using the summary
messages_task = Task(
    description=(
        "Draft a professional email, twitterd DM message, and linkedin message to the client based on the summarized content from 'summary_output.txt'. Ensure the message is clear, concise, and includes a call to action, mentioning the full report is attached. Add a relevant movie quote."
    ),
    expected_output='A well-crafted set of drafts as email, twitter DM and Linkedin message to the client.',
    tools=[summarization_tool],
    agent=message_drafter,
    input_file='summary_output.txt',
    output_file='messages_draft.txt'  # Specify the output file for the email draft
)    
# Define the crew with multiple agents and the tasks
consulting_crew = Crew(
    agents=[data_analyst_agent, research_specialist_agent, lead_consultant_agent, message_drafter],
    tasks=[data_analysis_task, research_task, lead_consultant_task, summarization_task, messages_task],
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True,
    Memory=True
)


# Client problem picked up from another source, can be integrated within later.
client_problem = {
    'problem_statement': 'We are facing declining sales in our main product line.',
    'key_issues': 'Increased competition, outdated marketing strategies.',
    'desired_outcome': 'Revitalize sales and regain market share.',
    'background_information': 'Our main product has been on the market for 10 years.',
    'timeline': 'Need results within the next 6 months.',
    'additional_information': 'Open to exploring new marketing channels.'
}


# Kickoff the process with the sample client problem statement
result = consulting_crew.kickoff(inputs={'client_problem': client_problem})


print(result)


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 1. Collect and clean data from various sources relevant to the client's market and competition using {'problem_statement': 'We are facing declining sales in our main product line.', 'key_issues': 'Increased competition, outdated marketing strategies.', 'desired_outcome': 'Revitalize sales and regain market share.', 'background_information': 'Our main product has been on the market for 10 years.', 'timeline': 'Need results within the next 6 months.', 'additional_information': 'Open to exploring new marketing channels.'} as context.
2. Perform statistical analyses and data modeling to identify trends and insights wherever possible.
3. Create visualizations to interpret data trends and provide data-driven insights wherever possible.
4. Collaborate with the Research Specialist and Lead Consultant to integrate findings into strategic recommendations.


> Entering new CrewAgentExecutor chain...
I need to delegate the tasks to the 